<a href="https://colab.research.google.com/github/hings/deep-learning/blob/main/%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5(%EC%A0%81%EC%9D%80%EB%8D%B0%EC%9D%B4%ED%84%B0_%2B_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%A6%9D%EB%8C%80_%2B_%EC%82%AC%EC%A0%84_%ED%95%99%EC%8A%B5_%EB%AA%A8%EB%8D%B8)_%EC%B5%9C%EC%A2%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from tensorflow import keras
from tensorflow.keras import layers
from keras import models
import os
import zipfile
import shutil
from os import path
from keras import optimizers
from keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
height=150
width=150

아래 실행후 kaggle.json 파일 선택하기

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"siri9108","key":"3d9e511fac2725d9c872900e7f4c98d0"}'}

In [ ]:
!mkdir ~p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle competitions download -c dogs-vs-cats

 99% 540M/543M [00:03<00:00, 164MB/s]
100% 543M/543M [00:03<00:00, 146MB/s]
 96% 261M/271M [00:02<00:00, 95.2MB/s]
100% 271M/271M [00:02<00:00, 112MB/s] 
  0% 0.00/86.8k [00:00<?, ?B/s]
100% 86.8k/86.8k [00:00<00:00, 92.5MB/s]


In [ ]:
local_zip = '/content/train.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
!mkdir dataset1
!mkdir dataset1/cat
!mkdir dataset1/dog

train_list = list(os.listdir('train'))
for img in train_list:
    if 'cat' in img:
        shutil.move ('train/'+img,'dataset1/cat/'+img)
    elif 'dog' in img:
        shutil.move ('train/'+img,'dataset1/dog/'+img)

In [ ]:
!mkdir dataset
!mkdir dataset/train
!mkdir dataset/test
!mkdir dataset/train/cat
!mkdir dataset/train/dog
!mkdir dataset/test/cat
!mkdir dataset/test/dog

In [ ]:
cats = list(os.listdir('dataset1/cat'))
dogs = list(os.listdir('dataset1/dog'))

In [ ]:
for i in range(1000):
    img = cats[i]
    shutil.move ('dataset1/cat/'+img,'dataset/train/cat/'+img)

In [ ]:
for i in range(1000, 1500):
    img = cats[i]
    shutil.move ('dataset1/cat/'+img,'dataset/test/cat/'+img)

In [ ]:
for i in range(1000):
    img = dogs[i]
    shutil.move ('dataset1/dog/'+img,'dataset/train/dog/'+img)

In [ ]:
for i in range(1000, 1500):
    img = dogs[i]
    shutil.move ('dataset1/dog/'+img,'dataset/test/dog/'+img)

In [ ]:
base_dir = '/content/dataset/'

train_dir = os.path.join(base_dir, 'train')
var_dir = os.path.join(base_dir, 'test')

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )

train_generator = train_datagen.flow_from_directory(train_dir,
                                                  batch_size=32,
                                                  target_size=(150, 150))
validation_generator =  test_datagen.flow_from_directory(var_dir,
                                                       batch_size=32,
                                                       target_size = (150, 150))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

In [ ]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))

In [ ]:
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
model.compile(optimizer=optimizers.RMSprop(lr=1e-5),
             loss='categorical_crossentropy',
            metrics = ['accuracy'])

In [ ]:
model.fit(train_generator, validation_data=validation_generator, epochs=20)

Epoch 1/20
63/63 [==============================] - 19s 307ms/step - loss: 0.5674 - accuracy: 0.7135 - val_loss: 0.3731 - val_accuracy: 0.8450
Epoch 2/20
63/63 [==============================] - 19s 306ms/step - loss: 0.4140 - accuracy: 0.8165 - val_loss: 0.2759 - val_accuracy: 0.8860
Epoch 3/20
63/63 [==============================] - 19s 303ms/step - loss: 0.3635 - accuracy: 0.8360 - val_loss: 0.2428 - val_accuracy: 0.8970
Epoch 4/20
63/63 [==============================] - 19s 307ms/step - loss: 0.3203 - accuracy: 0.8590 - val_loss: 0.2302 - val_accuracy: 0.9030
Epoch 5/20
63/63 [==============================] - 19s 299ms/step - loss: 0.2889 - accuracy: 0.8740 - val_loss: 0.2169 - val_accuracy: 0.9120
Epoch 6/20
63/63 [==============================] - 19s 302ms/step - loss: 0.2675 - accuracy: 0.8830 - val_loss: 0.2335 - val_accuracy: 0.9110
Epoch 7/20
63/63 [==============================] - 19s 307ms/step - loss: 0.2594 - accuracy: 0.8840 - val_loss: 0.2248 - val_accuracy: 0.9180

구글 로그인 후 버튼 클릭 코드를 붙여넣어서 엔터누르면 드라이브와 연동완료

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [1]:
cd /content/gdrive/My Drive

[Errno 2] No such file or directory: '/content/gdrive/My Drive'
/content


In [ ]:
!mkdir dog,cat
!mkdir dog
!mkdir cat

In [ ]:
cd -

In [ ]:
class_names = ['cat', 'dog']

file_list = os.listdir('./gdrive/My Drive/dog,cat')
for f in file_list:
    dataset_path = f
    data_dir = os.path.join('./gdrive/My Drive/dog,cat', dataset_path)

    img = keras.preprocessing.image.load_img(
        data_dir, target_size=(height, width)
    )
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    before = os.path.join('./gdrive/My Drive/dog,cat', dataset_path)
    update_cat = os.path.join('./gdrive/My Drive/cat')
    update_dog = os.path.join('./gdrive/My Drive/dog')

    if(class_names[np.argmax(score)] == 'cat'):
        src = before
        dir = update_cat
        shutil.move(src, dir)

    if(class_names[np.argmax(score)] == 'dog'):
        src = before
        dir = update_dog
        shutil.move(src, dir)

28


NameError: ignored